# Eat Safe, Love

## Notebook Set Up

In [6]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")
# Display the first document in the results using pprint
first_result = establishments.find_one(query)
pprint(first_result)

In [ ]:
import pandas as pd

Define the query to find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

Fetch all matching documents and convert them to a list
results = list(establishments.find(query))

Convert the list of results to a Pandas DataFrame
df = pd.DataFrame(results)

Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

Display the first 10 rows of the DataFrame
df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
from pprint import pprint

# Define the query to find establishments in London with a RatingValue >= 4
query = {
    "RatingValue": {"$gte": 4},  # RatingValue greater than or equal to 4
    "LocalAuthorityName": {"$regex": "London"}  # Use $regex to match any Local Authority Name containing 'London'
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with RatingValue >= 4 in London: {count}")

# Display the first document in the results using pprint
first_result = establishments.find_one(query)
pprint(first_result)



In [ ]:


# Define the query to find establishments in London with a RatingValue >= 4
query = {
    "RatingValue": {"$gte": 4},
    "LocalAuthorityName": {"$regex": "London"}
}

# Fetch all matching documents and convert them to a list
results = list(establishments.find(query))

# Convert the list of results to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
df.head(10)



### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
from pprint import pprint

# Define Penang Flavours' location
penang_latitude = 51.490142
penang_longitude = 0.083840

# Define the search range (0.01 degrees on either side of latitude and longitude)
degree_search = 0.01
latitude_range = {"$gte": penang_latitude - degree_search, "$lte": penang_latitude + degree_search}
longitude_range = {"$gte": penang_longitude - degree_search, "$lte": penang_longitude + degree_search}

# Define the query
query = {
    "RatingValue": 5,
    "geocode.latitude": latitude_range,
    "geocode.longitude": longitude_range
}

# Define the sort order (by hygiene score in ascending order)
sort = [("scores.Hygiene", 1)]

# Limit to the top 5 results
limit = 5

# Fetch the results
results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    pprint(result)



In [ ]:
# Convert result to Pandas DataFrame

# Define Penang Flavours' location
penang_latitude = 51.490142
penang_longitude = 0.083840

# Define the search range (0.01 degrees on either side of latitude and longitude)
degree_search = 0.01
latitude_range = {"$gte": penang_latitude - degree_search, "$lte": penang_latitude + degree_search}
longitude_range = {"$gte": penang_longitude - degree_search, "$lte": penang_longitude + degree_search}

# Define the query
query = {
    "RatingValue": 5,
    "geocode.latitude": latitude_range,
    "geocode.longitude": longitude_range
}

# Define the sort order (by hygiene score in ascending order)
sort = [("scores.Hygiene", 1)]

# Limit to the top 5 results
limit = 5

# Fetch the results and convert to a list
results = list(establishments.find(query).sort(sort).limit(limit))

# Convert the results to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
df.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create the aggregation pipeline
pipeline = [
    # Step 1: Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # Step 2: Group by LocalAuthorityName and count the number of establishments
    {"$group": {
        "_id": "$LocalAuthorityName",  # Group by Local Authority
        "count": {"$sum": 1}           # Count the number of documents
    }},
    
    # Step 3: Sort by count in descending order (highest to lowest)
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of local authorities with hygiene score of 0: {len(results)}")

# Print the first 10 results
pprint(results[:10])


In [ ]:
# Create the aggregation pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Step 1: Match establishments with a hygiene score of 0
    {"$group": {
        "_id": "$LocalAuthorityName",    # Step 2: Group by LocalAuthorityName
        "count": {"$sum": 1}             # Count the number of establishments per Local Authority
    }},
    {"$sort": {"count": -1}}             # Step 3: Sort by count in descending order (highest to lowest)
]

# Execute the aggregation pipeline and convert the result to a list
results = list(establishments.aggregate(pipeline))

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
df.head(10)
